# Amazon Bedrock Batch Inference

## 情報
- https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html
- 2024.1.25
    - Public Preview
    - 利用方法
        - REST API: あると思うが面倒
        - CLI: 無さそう
        - SDK: プレビューのがある
            - PythonとJavaのみ
            - https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
        - コンソール: 無さそう

## 検証内容
- Bedrockのbatch推論のquotaがon demandと関係ない（だからquotaに引っ掛からなくてうれしい）と言う説
- text to image (SDXL or Titan or Both)で500枚くらいバッチ推論ジョブを発行してどんな風に実行が完了するか（もしくは完了しないか）を見てみる
- On demandのquotaより明らかに早かったら嬉しい

## 検証

### 準備

In [1]:
! wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip

--2024-01-25 00:47:13--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 99.84.216.107, 99.84.216.166, 99.84.216.85, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|99.84.216.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314169135 (300M) [application/zip]
Saving to: ‘bedrock-python-sdk-reinvent.zip’

bedrock-python-sdk- 100%[===================>] 299.61M  35.6MB/s    in 7.3s    

2024-01-25 00:47:21 (40.8 MB/s) - ‘bedrock-python-sdk-reinvent.zip’ saved [314169135/314169135]



In [2]:
! unzip -d sdk bedrock-python-sdk-reinvent.zip

Archive:  bedrock-python-sdk-reinvent.zip
   creating: sdk/reviews/
  inflating: sdk/.unit-crt           
  inflating: sdk/botocore-1.32.4.tar.gz  
  inflating: sdk/AWSCLI32PY3.msi     
  inflating: sdk/awscli-1.30.4-py3-none-any.whl  
  inflating: sdk/AWSCLI64PY3.msi     
  inflating: sdk/AWSCLI64.msi        
  inflating: sdk/boto3-1.29.4.tar.gz  
  inflating: sdk/.unit               
  inflating: sdk/botocore-1.32.4-py3-none-any.whl  
  inflating: sdk/boto3-1.29.4-py3-none-any.whl  
  inflating: sdk/AWSCLI32.msi        
  inflating: sdk/awscli-1.30.4.tar.gz  
  inflating: sdk/.functional         
  inflating: sdk/models-starfort-report.json  
  inflating: sdk/manifest.json       
  inflating: sdk/.functional-crt     
  inflating: sdk/awscli-bundle.zip   
  inflating: sdk/AWSCLISetup.exe     
  inflating: sdk/reviews/botocore-1.32.4.commits  
  inflating: sdk/reviews/boto3-1.29.4.commits  
  inflating: sdk/reviews/botocore-1.32.4.html  
  inflating: sdk/reviews/awscli-1.30.4.html  
  